<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-задачи" data-toc-modified-id="Описание-задачи-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание задачи</a></span><ul class="toc-item"><li><span><a href="#Постановка-задачи" data-toc-modified-id="Постановка-задачи-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Постановка задачи</a></span></li><li><span><a href="#Ход-исполнения" data-toc-modified-id="Ход-исполнения-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Ход исполнения</a></span></li></ul></li><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обзор данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-инструментов" data-toc-modified-id="Загрузка-инструментов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Загрузка инструментов</a></span></li><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Загрузка данных</a></span></li></ul></li><li><span><a href="#Подготовка-наборов" data-toc-modified-id="Подготовка-наборов-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка наборов</a></span></li><li><span><a href="#Модели" data-toc-modified-id="Модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Модели</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Подготовка-наборов" data-toc-modified-id="Подготовка-наборов-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Подготовка наборов</a></span></li><li><span><a href="#Модели-на-полном-наборе" data-toc-modified-id="Модели-на-полном-наборе-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Модели на полном наборе</a></span></li><li><span><a href="#Модели-на-downsampling-наборе" data-toc-modified-id="Модели-на-downsampling-наборе-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Модели на downsampling-наборе</a></span></li><li><span><a href="#Модели-на-BERT-эмбеддингах-(DistilBertModel)" data-toc-modified-id="Модели-на-BERT-эмбеддингах-(DistilBertModel)-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Модели на BERT-эмбеддингах (DistilBertModel)</a></span></li><li><span><a href="#Sanity-Check" data-toc-modified-id="Sanity-Check-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Sanity Check</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# КЛАССИФИКАЦИЯ ТОКСИЧНОСТИ ТЕКСТА

## Описание задачи

### Постановка задачи

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. Заказчик представил набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества F1 не меньше 0.75.

### Ход исполнения

1. Загрузим и исследуем данные.
2. Проанализируем данные.
3. Обучим разные модели, включая BERT.
4. Проверим модели.

## Обзор данных

### Загрузка инструментов

In [1]:
!pip install torch
!pip install transformers
!pip install tqdm

In [2]:
# общие библиотеки
import pandas as pd
import numpy as np
import seaborn as sns

In C:\Users\1\anaconda3\envs\praktikum\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\1\anaconda3\envs\praktikum\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\1\anaconda3\envs\praktikum\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\1\anaconda3\envs\praktikum\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\1\anaconda3\envs\praktikum\lib\site-

In [3]:
# модели
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier 
from sklearn.dummy import DummyClassifier


# нейросети
import torch
import transformers

# скоринг
from sklearn.metrics import f1_score

# утилиты
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [4]:
# классическое преобразование текста
import nltk
from nltk.corpus import stopwords
import re

In [5]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
# настройки
import warnings

from tqdm import notebook
from tqdm import tqdm
from pprint import pprint
from time import time

sns.set_style('darkgrid')
nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings('ignore', category=RuntimeWarning)
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Описание данных

Описание данных предоставлено заказчиком.

Столбец `text` в содержит текст комментария, а `toxic` — целевой признак.

### Загрузка данных

In [7]:
# читаем файл и сохраняем в переменные дальнейшей работы, в том числе изменений
# используем оператор обработки исключений, чтобы избежать ошибки, при открытии данных,
# расположенных по другому адресу 
try: 
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('datasets/toxic_comments.csv')
                         
# создадим дополнительные переменные, для контроля изменений    
try: 
    df_ctrl = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df_ctrl = pd.read_csv('datasets/toxic_comments.csv')

In [8]:
# проверим работу данных
df.sample(5, random_state=42)

,text,toxic
119105,"Geez, are you forgetful! We've already discus...",0
131631,Carioca RFA \n\nThanks for your support on my ...,0
125326,"""\n\n Birthday \n\nNo worries, It's what I do ...",0
111256,Pseudoscience category? \n\nI'm assuming that ...,0
83590,"(and if such phrase exists, it would be provid...",0


Изучим данные более внимательно.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Проверим данные на дубликаты.

In [10]:
sum(df['text'].duplicated())

0

**Итоги:**

1. Данные загружены и функционируют.
2. Пропуски отсутствуют.
3. Тип данных соответствует содержанию колонок.

## Подготовка наборов

Создадим корпус текстов, приведем их все к нижнему регистру и необходимой кодировке.

In [11]:
df['text'] = df['text'].str.lower()

In [12]:
corpus = df['text']

In [13]:
X = corpus
y = df['toxic']

Изучим распределение классов в целевом признаке.

In [14]:
df['toxic'].value_counts(normalize=True).to_frame()

,toxic
0,0.898321
1,0.101679


**Итоги:**

Набор содержит только 10% положительных классов. Это необходимо учесть при обучении моделей.

## Модели

### Подготовка данных

Получим леммы, избавимся от лишних символов, а также преобразуем текст.

In [15]:
def text_preprocessing(text):
    tokenized = nltk.word_tokenize(text)
    joined = ' '.join(tokenized)
    text_only = re.sub(r"[^a-z0-9!@#\$%\^\&\*_\-,\.' ]", ' ', joined)
    final = ' '.join(text_only.split())
    return final

In [16]:
tqdm.pandas() 
df['token_text'] = df['text'].progress_apply(text_preprocessing)

100%|██████████| 159571/159571 [01:57<00:00, 1355.09it/s]


Выделим и преобразуем в отдельный массив лемматизированный текст.

In [17]:
lemms = df['token_text']

**Итоги:**

1. Мы получили леммы слов, а также подготовили данные для дальнейшего машинного обучения. 
2. Наблюдается сильный дисбаланс классов, поэтому попробуем два способа борьбы с ним:
- downsampling
- class_weight = balance
3. Проверим работу трех моделей: логистической регрессии, случайного леса и SGD классификатора.

### Подготовка наборов

In [18]:
features = 'token_text'
target = 'toxic'

In [19]:
X = df[features]
y = df[target]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

Подготовим downsampling выборку.

In [21]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_sample = features_zeros.sample(frac=0.1, random_state=42)
    target_sample = target_zeros.sample(frac=0.1, random_state=42)
    
    features_downsampled = pd.concat([features_sample] + [features_ones])
    target_downsampled = pd.concat([target_sample] + [target_ones])
    
    features_downsampled = shuffle(features_downsampled, random_state=42)
    target_downsampled = shuffle(target_downsampled, random_state=42)
    
    return features_downsampled, target_downsampled

In [22]:
X_train_dwnsmpld, y_train_dwnsmpld = downsample(X_train, y_train, 0.1)

Проверим размеры получившихся наборов и распределение классов.

In [23]:
len(X_train_dwnsmpld), len(y_train_dwnsmpld)

(27503, 27503)

In [24]:
y_train_dwnsmpld.value_counts(normalize=True).to_frame()

,toxic
1,0.530924
0,0.469076


**Итоги:**

1. Подготовили поные обучающий и тестовый наборы.
2. Подготовили наборы с уменьшенным количеством классов, чтобы избежать дисбаланса классов.

### Модели на полном наборе

In [25]:
models_score = {}
models_score['model_name'] = []
models_score['f1_score'] = []

In [26]:
logr_mod = LogisticRegression(solver="liblinear", random_state=42, class_weight='balanced')
rfc_mod = RandomForestClassifier(n_estimators=10, random_state=42, class_weight='balanced')
sgb_mod = SGDClassifier(max_iter = 1000, random_state=42, loss='modified_huber', class_weight='balanced')

In [27]:
def ml_slave(estimators, X_train, X_test, y_train, y_test, name):
    models_score['model_name'].append(name)
    
    count_tf_idf = TfidfVectorizer(stop_words=stopwords)
    tf_idf_train = count_tf_idf.fit_transform(X_train)
    tf_idf_test = count_tf_idf.transform(X_test)
    
    voting_clf = VotingClassifier(
        estimators=estimators,
        voting='soft', verbose=True)
    
    voting_clf.fit(tf_idf_train, y_train)
    predict = voting_clf.predict(tf_idf_test)
    f_score = round(f1_score(predict, y_test), 4)
    models_score['f1_score'].append(f_score)
    print(f'Итоговый f1 score: {f_score}')

In [28]:
%%time
ml_slave([('logr', logr_mod), ('rfc', rfc_mod), ('sgb', sgb_mod)], 
         X_train, X_test, y_train, y_test, 
         'VotingClassifier (LR, RFC, SGB) на основе полного набора')

[Voting] ..................... (1 of 3) Processing logr, total=   2.1s
[Voting] ...................... (2 of 3) Processing rfc, total= 1.9min
[Voting] ...................... (3 of 3) Processing sgb, total=   0.6s
Итоговый f1 score: 0.7899
Wall time: 2min 6s


**Итоги:**

Обобщение через VotingClassifier показало проходной результат меры f1=0.79. Что является допустимым.

### Модели на downsampling-наборе

In [29]:
logr_mod = LogisticRegression(solver="liblinear", random_state=42)
rfc_mod = RandomForestClassifier(n_estimators=10, random_state=42)
sgb_mod = SGDClassifier(max_iter = 1000, random_state=42, loss='modified_huber')

In [30]:
%%time
ml_slave([('logr', logr_mod), ('rfc', rfc_mod), ('sgb', sgb_mod)],
         X_train_dwnsmpld, X_test, y_train_dwnsmpld, y_test, 
         'VotingClassifier на основе downsampling-набора')

[Voting] ..................... (1 of 3) Processing logr, total=   0.2s
[Voting] ...................... (2 of 3) Processing rfc, total=   7.4s
[Voting] ...................... (3 of 3) Processing sgb, total=   0.1s
Итоговый f1 score: 0.648
Wall time: 10.2 s


**Итоги:**

Обобщение через VotingClassifier по downsampling-набору показало результат меры f1=0.66. Что значительно хуже полной выборки и не проходит по требованиям.

### Модели на BERT-эмбеддингах (DistilBertModel)

Проверим результаты при работе на модели DistilBert. При этом, используем только часть данных, чтобы сократить время обучения. Что, несомненно, скажется на результате.

Подготовим набор со сбалансированным количеством классов.

In [31]:
X_train_b = X_train_dwnsmpld.sample(300, random_state=42)
y_train_b =  df.iloc[X_train_b.index]['toxic']

# проверим результат
y_train_b.value_counts(normalize=True).to_frame()

,toxic
1,0.523333
0,0.476667


Классы распределны практически равномерно, выборки подготовлены.

In [32]:
# подготовим параметры
configuration = transformers.DistilBertConfig()
model = transformers.DistilBertModel(configuration)
pretrained_weights = 'distilbert-base-uncased'
tokenizer_class = transformers.DistilBertTokenizer

При этом как BERT так и DistilBert обучены работе только с предложениями длинной до 512 символов. Однако, в связи с необходимостью экономить GPU, сократим длину строки до 256. Несомненно, это также скажется на качестве обучения.

In [33]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

tokenized = X_train_b.apply(
    lambda x: tokenizer.encode(x[:256], add_special_tokens=True))

padded = np.array([i + [0]*(256 - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [34]:
# проверим результат
padded.shape, attention_mask.shape

((300, 256), (300, 256))

In [35]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
features = np.concatenate(embeddings)

  0%|          | 0/3 [00:00<?, ?it/s]

In [36]:
X_train_b = features

Подготовим тестовую выборку.

In [37]:
X_test_b = X_test.sample(200, random_state=42)
y_test_b =  df.iloc[X_test_b.index]['toxic']

In [38]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

tokenized = X_test_b.apply(
    lambda x: tokenizer.encode(x[:256], add_special_tokens=True))

padded = np.array([i + [0]*(256 - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [39]:
# проверим результат
padded.shape, attention_mask.shape

((200, 256), (200, 256))

In [40]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
features = np.concatenate(embeddings)

  0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
X_test_b = features

In [42]:
voting_clf = VotingClassifier(
        estimators=[('logr', logr_mod), ('rfc', rfc_mod), ('sgb', sgb_mod)],
        voting='soft', verbose=True)
    
voting_clf.fit(X_train_b, y_train_b)
predict = voting_clf.predict(X_test_b)
f_score = round(f1_score(predict, y_test_b), 4)
models_score['model_name'].append('VotingClassifier на основе BERT-эмбеддингов')
models_score['f1_score'].append(f_score)
print(f'Итоговый f1 score: {f_score}')

[Voting] ..................... (1 of 3) Processing logr, total=   0.2s
[Voting] ...................... (2 of 3) Processing rfc, total=   0.1s
[Voting] ...................... (3 of 3) Processing sgb, total=   0.0s
Итоговый f1 score: 0.2526


In [43]:
# очистим кэш
with torch.no_grad():
    torch.cuda.empty_cache()

**Итоги:**

Модель ожидаемо продемонстрировала показала метрику f1 ниже целевой (при учете сокращенной длины комментариев в два раза, а также сокращенной выборки в 300 наблюдений).

При этом, подобная предобработка требует значительных GPU мощностей и затрат времени.

### Sanity Check

Проверим адекватность моделей на основе классификатора, предсказывающего везде токсичный исход.

In [44]:
dummy = DummyClassifier(random_state=42, strategy='constant', constant=1)

In [45]:
ml_slave([('dm', dummy)], 
         X_train, X_test, y_train, y_test, 
         'Константная модель на основе DummyClassifier (constant=1)')

[Voting] ....................... (1 of 1) Processing dm, total=   0.0s
Итоговый f1 score: 0.1846


**Итоги:**

Константная модель показала наихудшую метрику f1, что подтверждает эффективность применения алгоритмов МО.

## Выводы

In [46]:
score_table = pd.DataFrame(models_score).sort_values(by='f1_score', ascending=False)
score_table['approved'] = score_table['f1_score'].apply(lambda x: 'yes' if x >= 0.75 else 'not')
score_table.style.highlight_max('f1_score', color = 'green', axis = 0)

,model_name,f1_score,approved
0,"VotingClassifier (LR, RFC, SGB) на основе полного набора",0.789900,yes
1,VotingClassifier на основе downsampling-набора,0.648000,not
2,VotingClassifier на основе BERT-эмбеддингов,0.252600,not
3,Константная модель на основе DummyClassifier (constant=1),0.184600,not


Для анализа тональности текста, нами был выбран ансамбль моделей (LogisticRegression, RandomForestClassifier, SGBClassifier). 

Наилучший результат целевой метрики был достигнут при использовании полной выборки, с балансированным весом классов. При этом, данная модель показала хорошие характеристики по времени обучения.

Кроме того, ансамбль был обучен на следующих наборах:
- downsampling-набор;
- BERT-эмбеддинги;
- константная модель.

Модели, обученные на этих наборах не прошли по значению целевой метрики. Кроме того, константная модель продемонстрировала самую низкую оценку, что подтверждает sanity check остальных моделей.